## column extraction

In [ ]:
# -*- coding: utf-8 -*-
"""
Eve OCR → Gemini → Parsed CSV Pipeline
Version: ladder pricing + bulk-pack mapping + avg price + unit split

- ดึง JSON จาก Gemini ตาม OCR text
- แยก unit จาก product_name สำหรับ pattern พวก "1,000 มล. x 2"
- ใช้ราคา step2 (1 แพ็ค / 1 ชิ้น / 1 กล่อง / 1 ขวด / 1 ซอง) เป็น discounted_price เสมอ
- โปรยกลัง (เช่น 10 แพ็ค, 1 ลัง ฯลฯ) จะเก็บ step1_avg_price ไว้ และ
  ถ้า discounted_price ยังว่าง → ใช้ step1_avg_price แทน
- ถ้าเป็นโปรยกลัง แต่ LLM ไม่ใส่ step1_avg_price ให้ → คำนวณให้เองจาก step1_price / จำนวนใน step1_qty
- กรณีพิเศษ:
  - ถ้าโปร 1 หน่วยอยู่ใน step1 (แต่ step2 ว่าง) → ใช้ step1_price เป็น discounted_price
  - ถ้ามี step1_avg_price อยู่แล้ว ให้ถือว่าเป็นดีลยกลัง/หลายชิ้น (แม้ step1_qty จะไม่เข้า pattern)
"""

import os
import glob
import json
import time
import random
import re
from typing import List, Dict, Tuple

import pandas as pd
from google import genai


# ===================== CONFIG =====================

BASE_DIR = r"C:\projecteve\OCR"

PREFERRED_INPUT_FILE = (
    r"C:\projecteve\OCR\data\yolo_crop\output_detail_only2025-11-03\ocr_results_2025-11-03.csv"
)

TEXT_COL_NAME = "text"
ITEMS_PER_BLOCK = 20

KEY_POOL = [
    "Press Key Here",
    "Press Key Here",
]

MODEL = "gemini-2.0-flash"
MAX_RETRY_PER_BLOCK = 5
COOLDOWN_SEC_ON_FAIL = 3
PRINT_MODEL_PREVIEW = True


# ===================== FILE DISCOVERY =====================

def find_fallback_csv(base_dir: str) -> str:
    candidates = glob.glob(os.path.join(base_dir, "**", "*.csv"), recursive=True)
    if not candidates:
        raise FileNotFoundError("หาไฟล์ .csv ไม่เจอเลยใต้โปรเจกต์")

    ocr_like = [
        p for p in candidates
        if any(k in p.lower() for k in ["ocr", "easyocr", "llm", "extract"])
    ]
    pick_from = ocr_like if ocr_like else candidates
    pick_from.sort(key=lambda p: os.path.getmtime(p), reverse=True)

    chosen = pick_from[0]
    print("⚠️ ใช้ fallback CSV:", chosen)
    return chosen


def resolve_input_file() -> str:
    if os.path.isfile(PREFERRED_INPUT_FILE):
        print("📄 ใช้ไฟล์ที่ตั้งไว้:", PREFERRED_INPUT_FILE)
        return PREFERRED_INPUT_FILE
    print("❗ ไม่พบไฟล์ที่ตั้งไว้ล่วงหน้า:", PREFERRED_INPUT_FILE)
    return find_fallback_csv(BASE_DIR)


# ===================== LOAD =====================

def load_ocr_items(csv_path: str, text_col: str) -> List[str]:
    df = pd.read_csv(csv_path, encoding="utf-8-sig")
    if text_col not in df.columns:
        raise ValueError(f"ไม่พบคอลัมน์ {text_col} ในไฟล์ {csv_path}")
    df[text_col] = df[text_col].fillna("")
    items = [str(t).strip() for t in df[text_col] if str(t).strip()]
    print(f"✅ พบ {len(items)} ป้าย OCR")
    return items


def make_blocks(items: List[str], n: int):
    """แบ่ง items เป็น blocks ละ n ชิ้น พร้อม index global"""
    return [list(enumerate(items[i:i+n], start=i)) for i in range(0, len(items), n)]


# ===================== PROMPT =====================

def build_prompt_for_block(block_items: List[Tuple[int, str]]) -> str:
    lines = [f"[ITEM {i}]\n{text.strip()}\n" for i, text in block_items]
    ocr_section = "\n".join(lines)
    prompt = f"""
คุณคือระบบ Extractor ป้ายราคาสินค้า

อินพุต:
- OCR หลายป้าย แต่ละป้ายขึ้นต้นด้วย [ITEM n]

เอาต์พุต:
- JSON array เท่านั้น (ห้ามสรุป/ห้ามข้อความอื่น)
- 1 object ต่อ 1 [ITEM n]
- ถ้าไม่มีข้อมูล ให้ใส่ "" (string ว่าง)

โครงสร้าง object:
{{
  "product_ID": "",
  "product_name": "",
  "unit": "",
  "normal_price": "",
  "discounted_price": "",
  "step1_qty": "",
  "step1_price": "",
  "step1_avg_price": "",
  "step2_qty": "",
  "step2_price": "",
  "step3_qty": "",
  "step3_price": ""
}}

นิยามสำคัญ:
- product_name:
    - ให้ดึง "ชื่อสินค้าเต็ม" ตามที่ปรากฏบนป้าย โดยไม่ตัดคำและไม่สรุปให้สั้นลง
    - รวมทุกคำที่เป็นชื่อแบรนด์, รุ่น, ซีรีส์, รสชาติ, สูตร, กลิ่น, ลักษณะสินค้า
      เช่น "แช่แข็ง", "พร้อมดื่ม", "ชนิดเม็ด", "ชนิดผง", "สูตรเข้มข้น" ฯลฯ
    - ถ้าป้ายเขียนชื่อยาว ให้ใช้ทั้งชื่อยาวนั้น ห้ามลดเหลือคำสั้น ๆ เช่น "สบู่", "น้ำยาล้างจาน"
    - ห้ามแปลภาษา และห้ามเปลี่ยนคำเอง ให้ใช้ข้อความตาม OCR ให้มากที่สุด
    - ไม่ต้องใส่คำที่เกี่ยวกับราคา เช่น "ราคาปกติ", "ราคาพิเศษ", "กก.ละ", "ชิ้นละ"

- normal_price      = ราคาปกติ/ราคาขายป้ายปกติ ต่อ 1 หน่วย (เช่น ต่อ 1 แพ็ค)
- discounted_price  = ราคาลดแล้ว ต่อ 1 หน่วย (ถ้ามีโปรซื้อ 1 แพ็ค / 1 ชิ้น)
- step1_qty/step1_price:
    - ใช้เก็บโปรแบบ "ซื้อหลายหน่วยเป็นเซ็ต" หรือ "ยกลัง"
    - เช่น "10 แพ็ค 650", "1 ลัง (24 ขวด) 480" ฯลฯ
- step1_avg_price:
    - ใช้เฉพาะกรณีโปร "ยกลัง/คาร์ตัน/หีบ" หรือแบบหลายแพ็คที่ถือว่าเป็นลัง
    - เป็นราคาเฉลี่ยต่อ 1 หน่วย (ต่อ 1 แพ็ค/ขวด/ชิ้น แล้วแต่ประเภท)
- step2/step3:
    - ใช้สำหรับโปรขั้นบันได (เช่น 1 แพ็ค 69 / 2 แพ็ค 120 / 3 แพ็ค 165)

ข้อจำกัด:
- ห้ามเพิ่ม key อื่นนอกจากด้านบน
- ราคาทั้งหมดเป็นตัวเลขล้วน (ห้ามใส่ "บาท", "฿", ".-" ฯลฯ)
- ถ้าไม่ทราบให้ใส่ ""

OCR หลายป้าย:
{ocr_section}
"""
    return prompt.strip()


# ===================== GEMINI CALL =====================

def call_gemini_once(prompt: str, api_key: str, model: str):
    client = genai.Client(api_key=api_key)
    resp = client.models.generate_content(model=model, contents=[prompt])

    if getattr(resp, "text", None):
        return resp.text.strip()

    parts = []
    for cand in getattr(resp, "candidates", []):
        for part in getattr(cand.content, "parts", []):
            if getattr(part, "text", None):
                parts.append(part.text)

    return "\n".join(parts).strip()


def call_gemini_with_retry(prompt, pool, model, retry, cooldown):
    last_err = None
    for i in range(retry):
        key = random.choice(pool)
        print(f"🔑 ใช้ API key (attempt {i+1}/{retry})")
        try:
            return call_gemini_once(prompt, key, model)
        except Exception as e:
            print(f"❌ Gemini error {i+1}/{retry}: {e}")
            time.sleep(cooldown)
            last_err = e
    raise RuntimeError(f"Gemini failed after {retry} tries: {last_err}")


# ===================== PARSE JSON =====================

def parse_gemini_json(resp_text: str) -> List[Dict]:
    if not isinstance(resp_text, str):
        return []
    text = resp_text.strip()

    # ตัดบล็อก ```...``` ถ้ามี
    if "```" in text:
        first = text.find("```")
        last = text.rfind("```")
        inner = text[first+3:last] if last > first else text[first+3:]
        text = inner.strip()
        if text.lower().startswith("json"):
            text = text[4:].strip()

    payload = None
    start, end = text.find("["), text.rfind("]")
    if start != -1 and end != -1 and end > start:
        payload = text[start:end+1]
    else:
        s, e = text.find("{"), text.rfind("}")
        if s != -1 and e != -1 and e > s:
            payload = "[" + text[s:e+1] + "]"

    if payload is None:
        print("⚠️ ไม่พบ JSON [] หรือ {} ในข้อความตอบกลับ")
        return []

    try:
        data = json.loads(payload)
    except Exception as e:
        print("⚠️ JSON parse fail:", e)
        with open("gemini_raw_failed.json.txt", "w", encoding="utf-8") as f:
            f.write(resp_text)
        return []

    rows = []
    if isinstance(data, dict):
        data = [data]

    for r in data if isinstance(data, list) else []:
        if isinstance(r, dict):
            rows.append({
                "product_ID": r.get("product_ID", ""),
                "product_name": r.get("product_name", ""),
                "unit": r.get("unit", ""),
                "normal_price": r.get("normal_price", ""),
                "discounted_price": r.get("discounted_price", ""),
                "step1_qty": r.get("step1_qty", ""),
                "step1_price": r.get("step1_price", ""),
                "step1_avg_price": r.get("step1_avg_price", ""),
                "step2_qty": r.get("step2_qty", ""),
                "step2_price": r.get("step2_price", ""),
                "step3_qty": r.get("step3_qty", ""),
                "step3_price": r.get("step3_price", ""),
            })
    return rows


# ===================== CLEANING & RULES =====================

def clean_number_like(v):
    if not v:
        return ""
    s = str(v)
    for bad in ["บาท", "฿", "ชิ้นละ", "ต่อ", ".-", ",", " "]:
        s = s.replace(bad, "")
    return s.strip()


def split_unit_from_name(row):
    """
    ย้าย pattern แบบ "1,000 มล. x 2" จาก product_name → unit
    ถ้า unit เดิมว่างเท่านั้น
    """
    name = str(row.get("product_name", "")).strip()
    unit = str(row.get("unit", "")).strip()

    if unit:
        return row

    pattern = r"(\d[\d,\.]*\s*(?:มล\.?|ลิตร|กก\.?|กรัม|ซีซี|ml|ML|g|kg|KG|L|ล\.)\s*(?:[x×]\s*\d+)?)\s*$"
    m = re.search(pattern, name)
    if m:
        unit_text = m.group(1).strip()
        new_name = name[:m.start()].strip()
        row["product_name"] = new_name
        row["unit"] = unit_text

    return row


def is_bulk_deal(step1_qty: str, unit: str) -> bool:
    """
    เช็คว่าเป็นโปรแบบ 'ยกลัง/ยกแพ็คเยอะๆ' ไหม

    เกณฑ์:
      - มีคำว่า ลัง/คาร์ตัน/หีบ/ยกลัง ใน step1_qty หรือ unit  → ยกลังแน่นอน
      - หรือ step1_qty เป็นรูปแบบ '10 แพ็ค', '12 แพค', '8 pack' (>= 6 แพ็ค) → ยกลัง
      - หรือ step1_qty เป็นตัวเลขล้วน เช่น '10', '12' (>= 6) → ยกลัง (กรณี LLM ให้เลขเฉย ๆ)
    """
    text_all = f"{step1_qty or ''} {unit or ''}"

    for kw in ["ลัง", "คาร์ตัน", "หีบ", "ยกลัง"]:
        if kw in text_all:
            return True

    qty_str = (step1_qty or "").strip()

    m = re.match(r"^(\d+)\s*(แพ็ค|แพค|pack)\b", qty_str, flags=re.IGNORECASE)
    if m:
        try:
            count = int(m.group(1))
            if count >= 6:
                return True
        except ValueError:
            pass

    m2 = re.match(r"^(\d+)$", qty_str)
    if m2:
        try:
            count = int(m2.group(1))
            if count >= 6:
                return True
        except ValueError:
            pass

    return False


def compute_avg_from_step1(step1_qty: str, step1_price: str) -> str:
    """
    คำนวณราคาเฉลี่ยจาก step1_qty + step1_price
    เช่น "10 แพ็ค" + "650" → "65"
    หรือ "10" + "650" → "65"
    """
    if not step1_qty or not step1_price:
        return ""

    m = re.search(r"(\d+)", step1_qty)
    if not m:
        return ""

    try:
        count = int(m.group(1))
        if count <= 0:
            return ""
    except ValueError:
        return ""

    price_clean = clean_number_like(step1_price)
    if not price_clean:
        return ""

    try:
        total = float(price_clean)
        avg = total / count
    except ValueError:
        return ""

    if abs(avg - round(avg)) < 1e-6:
        return str(int(round(avg)))
    s = f"{avg:.2f}"
    s = s.rstrip("0").rstrip(".")
    return s


def fix_discount_and_steps(row):
    """
    - ถ้า step2_qty เป็น 1 หน่วย (1 แพ็ค / 1 ชิ้น / 1 กล่อง / 1 ขวด / 1 ซอง ...)
      และมี step2_price → ใช้ step2_price เป็น discounted_price เสมอ
      จากนั้นเคลียร์ step2_qty, step2_price

    - กรณีเพิ่ม: ถ้า step2 ว่าง แต่ step1_qty เป็น 1 หน่วย (1 แพ็ค / 1 ชิ้น / 1 กล่อง / 1 ขวด / 1 ซอง ...)
      และมี step1_price → ใช้ step1_price เป็น discounted_price แล้วเคลียร์ step1_qty, step1_price

    - กรณีเพิ่มอีก: ถ้ามี step1_avg_price อยู่ ให้ถือว่าเป็นดีลยกลัง/หลายชิ้นโดยอัตโนมัติ
      (แม้ step1_qty จะไม่เข้า pattern) → ไม่ล้าง step1_avg_price และใช้เป็น discounted_price ถ้ายังว่าง

    - ตรวจว่าเป็นโปรยกลังหรือไม่ด้วย is_bulk_deal + เงื่อนไขข้างบน
      - ถ้าเป็นโปรยกลัง:
          - ถ้า step1_avg_price ว่าง แต่มี step1_qty & step1_price → คำนวณ avg ให้เอง
          - ถ้ามี step1_avg_price แล้ว และ discounted_price ยังว่าง → ใช้ step1_avg_price เป็น discounted_price
          - ไม่เคลียร์ step1_avg_price ออก
      - ถ้าไม่ใช่โปรยกลัง:
          - เคลียร์ step1_avg_price = "" (ไม่ต้องมี avg)
    """
    dp = str(row.get("discounted_price", "")).strip()
    step1_qty = str(row.get("step1_qty", "")).strip()
    step1_price = str(row.get("step1_price", "")).strip()
    step1_avg = str(row.get("step1_avg_price", "")).strip()
    step2_qty = str(row.get("step2_qty", "")).strip()
    step2_price = str(row.get("step2_price", "")).strip()
    unit = str(row.get("unit", "")).strip()

    single_patterns = [
        "1แพ็ค", "แพ็ค1",
        "1ชิ้น", "ชิ้น1",
        "1กล่อง", "กล่อง1",
        "1ขวด", "ขวด1",
        "1ซอง", "ซอง1",
    ]

    qty2_no_space = step2_qty.replace(" ", "")
    is_single_step2 = any(p in qty2_no_space for p in single_patterns)

    if is_single_step2 and step2_price:
        dp = step2_price
        step2_qty = ""
        step2_price = ""

    if not step2_qty and not step2_price:
        qty1_no_space = step1_qty.replace(" ", "")
        is_single_step1 = any(p in qty1_no_space for p in single_patterns)
        if is_single_step1 and step1_price:
            dp = step1_price
            step1_qty = ""
            step1_price = ""

    bulk = is_bulk_deal(step1_qty, unit)

    if not bulk and step1_avg:
        bulk = True

    if bulk:
        if not step1_avg and step1_price and step1_qty:
            step1_avg = compute_avg_from_step1(step1_qty, step1_price)

        if not dp and step1_avg:
            dp = step1_avg
    else:
        step1_avg = ""

    row["discounted_price"] = dp
    row["step2_qty"] = step2_qty
    row["step2_price"] = step2_price
    row["step1_qty"] = step1_qty
    row["step1_price"] = step1_price
    row["step1_avg_price"] = step1_avg
    return row


# ===================== MAIN =====================

def main():
    csv_path = resolve_input_file()
    items = load_ocr_items(csv_path, TEXT_COL_NAME)
    blocks = make_blocks(items, ITEMS_PER_BLOCK)
    print(f"🔪 รวมเป็น {len(blocks)} blocks")

    all_rows = []

    for i, block in enumerate(blocks, 1):
        print(f"\n🚀 Block {i}/{len(blocks)} | ITEM index {block[0][0]}–{block[-1][0]}")
        prompt = build_prompt_for_block(block)
        resp_text = call_gemini_with_retry(
            prompt, KEY_POOL, MODEL, MAX_RETRY_PER_BLOCK, COOLDOWN_SEC_ON_FAIL
        )

        if PRINT_MODEL_PREVIEW:
            print("\n===== MODEL PREVIEW (first 600 chars) =====")
            print(resp_text[:600])
            print("\n===== END PREVIEW =====")

        rows = parse_gemini_json(resp_text)
        print(f"➡ ได้ {len(rows)} rows จาก block นี้")
        all_rows.extend(rows)

    if not all_rows:
        print("⚠️ ไม่ได้ rows จากทุก block เลย เลยยังไม่เขียนไฟล์ CSV (กันไฟล์ว่าง)")
        return

    df = pd.DataFrame(all_rows)

    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = (
                df[col]
                .astype(str)
                .str.replace(r"[\r\n]+", " ", regex=True)
                .str.replace(r"\s+", " ", regex=True)
                .str.strip()
            )

    df = df.apply(split_unit_from_name, axis=1)
    df = df.apply(fix_discount_and_steps, axis=1)

    for c in ["normal_price", "discounted_price", "step1_price", "step1_avg_price", "step2_price", "step3_price"]:
        if c in df.columns:
            df[c] = df[c].apply(clean_number_like)

    out = os.path.splitext(csv_path)[0] + "_parsed_all.csv"
    df.to_csv(out, index=False, encoding="utf-8-sig")
    print("\n🎉 DONE →", out)
    print(df.head(10))


if __name__ == "__main__":
    main()


📄 ใช้ไฟล์ที่ตั้งไว้: C:\projecteve\OCR\data\yolo_crop\output_detail_only2025-11-03\ocr_results_2025-11-03.csv
✅ พบ 195 ป้าย OCR
🔪 รวมเป็น 10 blocks

🚀 Block 1/10 | ITEM index 0–19
🔑 ใช้ API key (attempt 1/5)

===== MODEL PREVIEW (first 600 chars) =====
```json
[
  {
    "product_ID": "10031603",
    "product_name": "ข้าวหอมมะลิ 100%",
    "unit": "ถุง",
    "normal_price": "180",
    "discounted_price": "175",
    "step1_qty": "",
    "step1_price": "",
    "step1_avg_price": "",
    "step2_qty": "",
    "step2_price": "",
    "step3_qty": "",
    "step3_price": ""
  },
  {
    "product_ID": "",
    "product_name": "รับฟรี คูปองชิงโชค",
    "unit": "",
    "normal_price": "",
    "discounted_price": "",
    "step1_qty": "",
    "step1_price": "",
    "step1_avg_price": "",
    "step2_qty": "",
    "step2_price": "",
    "step3_qty": "",
    

===== END PREVIEW =====
➡ ได้ 20 rows จาก block นี้

🚀 Block 2/10 | ITEM index 20–39
🔑 ใช้ API key (attempt 1/5)

===== MODEL PREVIEW (first 600 